In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
import os
from pathlib import Path
from collections import defaultdict
import os
from argparse import Namespace
from joblib import dump, load
base_dir = Path('/Users/vinay/Projects/Recsys')
if not base_dir:
    base_dir = Path(os.getcwd())
from tqdm import tqdm

data_dir = base_dir/'data'/'archive'
store_dir = base_dir/'artifacts'
import wandb
from torch.utils.data import DataLoader as dl

In [186]:
df = pd.read_csv(data_dir/'full_rating.csv')

In [188]:
#make a sample df

df[df['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
170,1,8507,5.0,2004-09-10 03:13:47
171,1,8636,4.5,2005-04-02 23:44:53
172,1,8690,3.5,2005-04-02 23:33:15
173,1,8961,4.0,2005-04-02 23:47:09


In [194]:

def reduce_sz(df,sz=10000):
    all_users = sorted(list(set(df['userId'])))
    data = []
    start = 0
    for user in tqdm(all_users):
        temp = df[df['userId'] == user]
        end = start+len(temp)
        temp = copy.deepcopy(temp.reindex(range(start,end)))
        start = end
        data.append(temp)
        if end > sz:
            break
    return pd.concat(data)
        

In [195]:
sample_df = reduce_sz(df)

  0%|                                       | 90/138493 [00:01<31:08, 74.07it/s]


In [198]:
part_df = reduce_sz(df,sz=1000000)

  5%|█▊                                   | 6742/138493 [01:28<28:56, 75.85it/s]


In [199]:
sample_df.to_csv(data_dir/'sample_rating.csv',index=False)
part_df.to_csv(data_dir/'partial_rating.csv',index=False)

In [13]:
df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [35]:
from collections import Counter,defaultdict
user_freq_dict = Counter(df['userId'].tolist())

## Notes/Observations

- We have 20M `user-rating` data from 1995 to 2015.
- To avoid `cold-start` problem I choose to split each user data


In [16]:
df[['rating']].describe()

,rating
count,2.000026e+07
mean,3.525529e+00
std,1.051989e+00
min,5.000000e-01
25%,3.000000e+00
50%,3.500000e+00
75%,4.000000e+00
max,5.000000e+00


In [19]:
user_freq = pd.Series(user_freq_dict.values())
user_freq.min(),user_freq.mean(),user_freq.max()

(20, 144.4135299257002, 9254)

In [25]:
df['date'] = pd.to_datetime(df['timestamp'])


In [27]:
df.sort_values(by='date')

,userId,movieId,rating,timestamp,date
4182421,28507,1176,4.0,1995-01-09 11:46:44,1995-01-09 11:46:44
18950979,131160,1079,3.0,1995-01-09 11:46:49,1995-01-09 11:46:49
18950936,131160,47,5.0,1995-01-09 11:46:49,1995-01-09 11:46:49
18950930,131160,21,3.0,1995-01-09 11:46:49,1995-01-09 11:46:49
12341178,85252,45,3.0,1996-01-29 00:00:00,1996-01-29 00:00:00
...,...,...,...,...,...
7819902,53930,118706,3.5,2015-03-31 06:00:51,2015-03-31 06:00:51
2508834,16978,2093,3.5,2015-03-31 06:03:17,2015-03-31 06:03:17
12898546,89081,55232,3.5,2015-03-31 06:11:26,2015-03-31 06:11:26
12898527,89081,52458,4.0,2015-03-31 06:11:28,2015-03-31 06:11:28


In [37]:
t = df[df['userId'] == 1]
sz = len(t)
t

,userId,movieId,rating,timestamp,date
0,1,2,3.5,2005-04-02 23:53:47,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40,2005-04-02 23:29:40
...,...,...,...,...,...
170,1,8507,5.0,2004-09-10 03:13:47,2004-09-10 03:13:47
171,1,8636,4.5,2005-04-02 23:44:53,2005-04-02 23:44:53
172,1,8690,3.5,2005-04-02 23:33:15,2005-04-02 23:33:15
173,1,8961,4.0,2005-04-02 23:47:09,2005-04-02 23:47:09


In [38]:
a = slice(0,3)
l = [0,9,8,7,6,6]
l[a]

[0, 9, 8]

In [42]:
t = df.sample(frac=1,ran)
t[:10]

,userId,movieId,rating,timestamp,date
5872032,40421,2087,4.5,2008-10-28 13:10:55,2008-10-28 13:10:55
15384869,106410,56174,3.5,2008-06-03 13:38:21,2008-06-03 13:38:21
9867873,68189,2719,1.0,1999-10-16 23:39:10,1999-10-16 23:39:10
12561962,86773,2826,3.0,2013-02-22 11:35:30,2013-02-22 11:35:30
19092459,132069,2710,5.0,2000-08-20 03:31:17,2000-08-20 03:31:17
19743114,136694,231,4.0,2004-11-12 13:50:08,2004-11-12 13:50:08
14106139,97434,5954,3.0,2007-04-22 15:57:28,2007-04-22 15:57:28
7696389,52988,500,3.0,1996-06-10 15:02:20,1996-06-10 15:02:20
1709984,11555,110,5.0,2001-10-14 16:16:29,2001-10-14 16:16:29
3473142,23662,3763,4.0,2000-11-19 17:35:14,2000-11-19 17:35:14


In [45]:
t[a]['userId'].tolist()

[40421, 106410, 68189]

In [233]:
#df.to_csv(data_dir/'full_rating.csv',index=False)

In [238]:
df.iloc[[0,9,8]].reset_index()

,index,userId,movieId,rating,timestamp
0,0,1,2,3.5,2005-04-02 23:53:47
1,9,1,260,4.0,2005-04-02 23:33:46
2,8,1,253,4.0,2005-04-02 23:35:40


## Creating and storing the data-split.

In [11]:
global seed 
import random
seed = 0
def set_seeds(seed=0):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # multi-GPU
set_seeds(seed)
    
def get_data(data_dir,split:list,mode,data_type,save_to_disk=False):
    #split = [0.6,0.5]
    #mode = 'random' or 'seq_aware'
    trn,vld,tst = defaultdict(list),defaultdict(list),defaultdict(list)
    df = pd.read_csv(data_dir/(data_type+'_'+'rating.csv'))
    #df = df.sample(frac=1,random_state=seed)
    perm = random.sample(range(len(df)),len(df))
    df = df.iloc[perm].reset_index()
    df['date'] = pd.to_datetime(df['timestamp'])
    user_ids = set(df['userId'])
    for user in tqdm(user_ids):
        tmp = df[df['userId'] == user]
        if split == 'seq_aware':
            tmp = tmp.sort_values(by='date')
        sz = len(tmp)
        #splitting-ids
        t,v = int(split[0]*sz),int(split[0]*sz)+int((sz-int(split[0]*sz))*split[-1])
        trn_ids,vld_ids,tst_ids = slice(0,t),slice(t,v),slice(v,sz)
        
        trn['user_id'].extend(tmp[trn_ids]['userId'].tolist())
        trn['rating'].extend(tmp[trn_ids]['rating'].tolist())
        trn['movie_id'].extend(tmp[trn_ids]['movieId'].tolist())
        
        
        vld['user_id'].extend(tmp[vld_ids]['userId'].tolist())
        vld['rating'].extend(tmp[vld_ids]['rating'].tolist())
        vld['movie_id'].extend(tmp[vld_ids]['movieId'].tolist())
        
        tst['user_id'].extend(tmp[tst_ids]['userId'].tolist())
        tst['rating'].extend(tmp[tst_ids]['rating'].tolist())
        tst['movie_id'].extend(tmp[tst_ids]['movieId'].tolist())
    trn,vld,tst = pd.DataFrame(trn),pd.DataFrame(vld),pd.DataFrame(tst)
    if save_to_disk:
        trn.to_csv(data_dir/(data_type+'_'+mode+'_trn.csv'))
        vld.to_csv(data_dir/(data_type+'_'+mode+'_vld.csv'))
        tst.to_csv(data_dir/(data_type+'_'+mode+'_tst.csv'))
    return trn,vld,tst
        
    
        
            
            
            


    

In [240]:
t,v,tt = get_data(data_dir,[0.6,0.5],'random','sample')

100%|█████████████████████████████████████████| 91/91 [00:00<00:00, 1643.87it/s]


In [243]:
set(t['user_id'])== set(tt['user_id'])

True

In [246]:
len(set(tt['movie_id'])-set(t['movie_id']))/len(set(tt['movie_id']))

0.30165289256198347

We may be not able to use all this data.Let's make smaller data-sets

In [247]:
data_type = 'sample'
split = 'random'
t,v,tt = get_data(data_dir,[0.6,0.5],split,data_type,save_to_disk=True)

100%|█████████████████████████████████████████| 91/91 [00:00<00:00, 1564.59it/s]


In [248]:
data_type = 'partial'
t,v,tt = get_data(data_dir,[0.6,0.5],split,data_type,save_to_disk=True)

100%|██████████████████████████████████████| 6743/6743 [00:07<00:00, 921.97it/s]


In [211]:
len(set(t['user_id'])),len(set(t['movie_id']))

(6743, 12649)

In [213]:
assert set(t['user_id']) == set(tt['user_id'])
assert set(t['movie_id']) != set(tt['movie_id'])

In [249]:
len(set(tt['movie_id'])-set(t['movie_id']))/len(set(tt['movie_id']))

0.07414910858995137

Note that 7 percent of movies are not there in training data

## Testing on dummy data

In [12]:
class RecsysDataset(torch.utils.data.Dataset):
    def __init__(self,df,usr_dict=None,mov_dict=None):
        self.df = df
        self.usr_dict = usr_dict
        self.mov_dict = mov_dict
    def __getitem__(self,index):
        if self.usr_dict and self.mov_dict:
            return [self.usr_dict[int(self.df.iloc[index]['user_id'])],self.mov_dict[int(self.df.iloc[index]['movie_id'])]],self.df.iloc[index]['rating']
        else:
            return [int(self.df.iloc[index]['user_id']-1),int(self.df.iloc[index]['movie_id']-1)],self.df.iloc[index]['rating']
            
    def __len__(self):
        return len(self.df)
        

In [132]:
sample = pd.DataFrame({'user_id':[1,2,3,2,2,3,2,2],'movie_id':[1,2,3,3,3,2,1,1],'rating':[2.0,1.0,4.0,5.0,1.3,3.5,3.0,4.5]})
trn_ids = random.sample(range(8),4,)
valid_ids = [i for i in range(8) if i not in trn_ids]


In [133]:
import copy

In [134]:
sample_trn,sample_vld = copy.deepcopy(sample.iloc[trn_ids].reset_index()),copy.deepcopy(sample.iloc[valid_ids].reset_index())


In [135]:
sample_vld = RecsysDataset(sample_vld)
sample_trn = RecsysDataset(sample_trn)

In [170]:
sample_vld[0]


([0, 0], 2.0)

In [137]:
train_loader = torch.utils.data.DataLoader(sample_trn, batch_size=2, shuffle=True)

In [138]:
valid_loader = torch.utils.data.DataLoader(sample_vld, batch_size=2, shuffle=True)

In [139]:
for u,r in train_loader:
    #user,item = u
    print(f'user:{u[0]},item:{u[-1]} and rating:{r}')
    #print(u)
    break

user:tensor([1, 2]),item:tensor([2, 1]) and rating:tensor([1.3000, 3.5000], dtype=torch.float64)


In [140]:
embedding = nn.Embedding(3, 5)
a = torch.tensor([1,2,0])
embedding(a)

tensor([[-1.5350,  0.0490, -1.6023, -1.3283, -2.2907],
        [ 1.6614,  1.2145, -2.7802,  0.4113, -0.2785],
        [ 0.2881, -0.2528, -0.9804, -0.6115, -2.3585]],
       grad_fn=<EmbeddingBackward0>)

In [123]:
1-torch.clip(torch.tensor(9),min=0.0,max=1.0)

tensor(0)

## Model

In [13]:
class NCF(nn.Module):
    def __init__(self,user_sz,item_sz,embd_sz,dropout_fac,min_r=0.0,max_r=5.0,alpha=0.5,with_variable_alpha=False):
        super().__init__()
        self.dropout_fac = dropout_fac
        self.user_embd_mtrx = nn.Embedding(user_sz,embd_sz)
        self.item_embd_mtrx = nn.Embedding(item_sz,embd_sz)
        #bias = torch.zeros(size=(user_sz, 1), requires_grad=True)
        self.h =  nn.Linear(embd_sz,1)
        self.fst_lyr = nn.Linear(embd_sz*2,embd_sz)
        self.snd_lyr = nn.Linear(embd_sz,embd_sz//2)
        self.thrd_lyr = nn.Linear(embd_sz//2,embd_sz//4)
        self.out_lyr = nn.Linear(embd_sz//4,1)
        self.alpha = torch.tensor(alpha)
        self.min_r,self.max_r = min_r,max_r
        if with_variable_alpha:
            self.alpha = torch.tensor(alpha,requires_grad=True)
        
    def forward(self,x):
        user_emd = self.user_embd_mtrx(x[0])
        item_emd = self.item_embd_mtrx(x[-1])
        #hadamard-product
        gmf = user_emd*item_emd
        gmf = self.h(gmf)
        
        mlp = torch.cat([user_emd,item_emd],dim=-1)
        mlp = self.out_lyr(self.thrd_lyr(self.snd_lyr(F.dropout(self.fst_lyr(mlp),p=self.dropout_fac))))
        fac = torch.clip(self.alpha,min=0.0,max=1.0)
        out = fac*gmf+ (1-fac)*mlp
        out = torch.clip(out,min=self.min_r,max=self.max_r)
        return out
        
        

In [150]:
#does it work
model = NCF(3,3,4,0.5)
for u,r in train_loader:
    #user,item = u
    print(f'user:{u[0]},item:{u[-1]} and rating:{r}')
    #print(u)
    out = model(u)
    print(f'output of the network=> out:{out},shape:{out.shape}')
    break

user:tensor([1, 1]),item:tensor([2, 0]) and rating:tensor([1.3000, 4.5000], dtype=torch.float64)
output of the network=> out:tensor([[0.],
        [0.]], grad_fn=<ClampBackward1>),shape:torch.Size([2, 1])


## Trainer Class

In [14]:
class Trainer(object):
    def __init__(self, model, device,loss_fn=None, optimizer=None, scheduler=None,artifacts_loc=None,exp_tracker=None):

        # Set params
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.store_loc = artifacts_loc
        self.exp_tracker = exp_tracker

    def train_step(self, dataloader):
        """Train step."""
        # Set model to train mode
        self.model.train()
        loss = 0.0

        # Iterate over train batches
        for i, batch in enumerate(dataloader):
            #batch = [item.to(self.device) for item in batch]  # Set device
            inputs,targets = batch
            inputs = [item.to(self.device) for item in inputs]
            targets = targets.to(self.device)
            #inputs, targets = batch[:-1], batch[-1]
            #import pdb;pdb.set_trace()
            self.optimizer.zero_grad()  # Reset gradients
            z = self.model(inputs)  # Forward pass
            targets = targets.reshape(z.shape)
            J = self.loss_fn(z.float(), targets.float())  # Define loss
            J.backward()  # Backward pass
            self.optimizer.step()  # Update weights

            # Cumulative Metrics
            loss += (J.detach().item() - loss) / (i + 1)

        return loss

    def eval_step(self, dataloader):
        """Validation or test step."""
        # Set model to eval mode
        self.model.eval()
        loss = 0.0
        y_trues, y_probs = [], []

        # Iterate over val batches
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):
                inputs,y_true = batch
                inputs = [item.to(self.device) for item in inputs]
                y_true = y_true.to(self.device).float()

                # Step
                z = self.model(inputs).float()  # Forward pass
                y_true = y_true.reshape(z.shape)
                J = self.loss_fn(z, y_true).item()

                # Cumulative Metrics
                loss += (J - loss) / (i + 1)

                # Store outputs
                y_prob = z.cpu().numpy()
                y_probs.extend(y_prob)
                y_trues.extend(y_true.cpu().numpy())

        return loss, np.vstack(y_trues), np.vstack(y_probs)

    def predict_step(self, dataloader):
        """Prediction step."""
        # Set model to eval mode
        self.model.eval()
        y_probs = []

        # Iterate over val batches
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):

                # Forward pass w/ inputs
                inputs, targets = batch
                z = self.model(inputs).float()

                # Store outputs
                y_prob = z.cpu().numpy()
                y_probs.extend(y_prob)

        return np.vstack(y_probs)
    
    def train(self, num_epochs, patience, train_dataloader, val_dataloader, 
              tolerance=1e-5):
        best_val_loss = np.inf
        training_stats = defaultdict(list)
        for epoch in tqdm(range(num_epochs)):
            # Steps
            train_loss = self.train_step(dataloader=train_dataloader)
            val_loss, _, _ = self.eval_step(dataloader=val_dataloader)
            #store stats
            training_stats['epoch'].append(epoch)
            training_stats['train_loss'].append(train_loss)
            training_stats['val_loss'].append(val_loss)
            #log-stats
            if self.exp_tracker == 'wandb':
                log_metrics = {'epoch':epoch,'train_loss':train_loss,'val_loss':val_loss}
                wandb.log(log_metrics,step=epoch)
            
            self.scheduler.step(val_loss)

            # Early stopping
            if val_loss < best_val_loss - tolerance:
                best_val_loss = val_loss
                best_model = self.model
                _patience = patience  # reset _patience
            else:
                _patience -= 1
            if not _patience:  # 0
                print("Stopping early!")
                break

            # Tracking
            #mlflow.log_metrics({"train_loss": train_loss, "val_loss": val_loss}, step=epoch)

            # Logging
            if epoch%10 == 0:
                print(
                    f"Epoch: {epoch+1} | "
                    f"train_loss: {train_loss:.5f}, "
                    f"val_loss: {val_loss:.5f}, "
                    f"lr: {self.optimizer.param_groups[0]['lr']:.2E}, "
                    f"_patience: {_patience}"
                )
        if self.store_loc:
            pd.DataFrame(training_stats).to_csv(self.store_loc/'training_stats.csv',index=False)
        return best_model, best_val_loss

In [152]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=5)

trainer = Trainer(model,'cpu',loss_fn,optimizer,scheduler)

In [153]:
trainer.train(100,10,train_loader,valid_loader)

Epoch: 1 | train_loss: 14.79750, val_loss: 7.50000, lr: 1.00E-03, _patience: 10
Stopping early!


(NCF(
   (user_embd_mtrx): Embedding(3, 4)
   (item_embd_mtrx): Embedding(3, 4)
   (h): Linear(in_features=4, out_features=1, bias=True)
   (fst_lyr): Linear(in_features=8, out_features=4, bias=True)
   (snd_lyr): Linear(in_features=4, out_features=2, bias=True)
   (thrd_lyr): Linear(in_features=2, out_features=1, bias=True)
   (out_lyr): Linear(in_features=1, out_features=1, bias=True)
 ),
 7.5)

The code -> Network+trainingloop works. Let's write experiment tracking system.

In [253]:
os.listdir(data_dir)

['sample_random_tst.csv',
 'partial_rating.csv',
 'full_rating.csv',
 'partial_random_trn.csv',
 'sample_random_vld.csv',
 'tag.csv',
 'partial_random_tst.csv',
 'sample_rating.csv',
 'genome_scores.csv',
 'sample_random_trn.csv',
 'partial_random_vld.csv',
 'genome_tags.csv',
 'link.csv',
 'movie.csv']

In [15]:

#mode = 'train','inference'
#data_type = 'sample','full','partial'

args = Namespace(
    exp_tracker = 'wandb',
    base_dir = '/Users/vinay/Projects/Recsys',
    model_path = '',
    trail_id = 0,
    dataset = 'movielens',
    mode = 'train',
    on = ['trn','vld','tst'],
    alpha=0.5,
    with_variable_alpha=False,
    data_type = 'sample',
    split_type = 'random',
    split = [0.6,0.5],
    embd_sz = 10,
    batch_size=64, 
    dropout_p=0.5,
    lr=2e-4,
    num_epochs=100,
    patience=10
)



In [21]:
vars(args)

{'exp_tracker': 'wandb',
 'base_dir': '/Users/vinay/Projects/Recsys',
 'model_path': '',
 'trail_id': 0,
 'dataset': 'movielens',
 'mode': 'train',
 'on': ['trn', 'vld', 'tst'],
 'alpha': 0.5,
 'with_variable_alpha': False,
 'data_type': 'sample',
 'split_type': 'random',
 'split': [0.6, 0.5],
 'embd_sz': 10,
 'batch_size': 64,
 'dropout_p': 0.5,
 'lr': 0.0002,
 'num_epochs': 100,
 'patience': 10}

In [232]:
#sanity testing

set(t['movie_id']) != set(tt['movie_id'])

'sample_seq_aware_trn.csv'

In [19]:
def run(args):
    #main function that handles everything.
    config_dict = vars(args)
    if args.exp_tracker == 'wandb':
        wandb.init(project=f"{args.trail_id}_{args.dataset}_{args.data_type}",config=config_dict)
        
    base_dir = Path(args.base_dir)
    if not args.base_dir:
        base_dir = Path(os.getcwd())
        
    if args.dataset == 'movielens':
        data_dir = base_dir/'data'/'archive'
    else:
        Print('Unknown dataset')
        exit()
 
    store_dir = base_dir/'artifacts'
    base_dir = args.base_dir
    os.makedirs(store_dir,exist_ok=True)
    
    #Check if all the necessary data is already there
    is_data_premade = True
    for d_type in args.on:
        file_name = args.data_type+'_'+args.split_type+'_'+d_type+'.csv'
        if file_name not in os.listdir(data_dir):
            is_data_premade=False
   
    #load the dataset
    df = pd.read_csv(data_dir/(args.data_type+'_'+'rating.csv'))
    mov_dict = dict(zip(sorted(set(df['movieId'])),range(len(sorted(set(df['movieId']))))))
    usr_dict = dict(zip(sorted(set(df['userId'])),range(len(sorted(set(df['userId']))))))
        
    if not is_data_premade:
        pass
        #get-data
        #?
        #trn_df,vld_df,tst_df = get_data()
    else:
        #load the csv's
        trn_file= args.data_type+'_'+args.split_type+'_'+'trn'+'.csv'
        vld_file = args.data_type+'_'+args.split_type+'_'+'vld'+'.csv'
        tst_file = args.data_type+'_'+args.split_type+'_'+'tst'+'.csv'
        trn_df,vld_df,tst_df = pd.read_csv(data_dir/trn_file),pd.read_csv(data_dir/vld_file),pd.read_csv(data_dir/tst_file)
        
    train_artifacts_loc = store_dir/(str(args.trail_id)+'_'+args.dataset+'_'+args.data_type)
    exp_tracker = args.exp_tracker
    if args.mode == 'train':
         
        #train-from scratch
        trn = RecsysDataset(trn_df,usr_dict,mov_dict)
        vld = RecsysDataset(vld_df,usr_dict,mov_dict)
        tst = RecsysDataset(tst_df,usr_dict,mov_dict)
        
        trn_dl,vld_dl,tst_dl = dl(trn, batch_size=args.batch_size, shuffle=True),dl(vld, batch_size=args.batch_size),dl(tst, batch_size=args.batch_size)
        
        #get model
        emb_len_usr = len(set(df['userId']))
        emb_len_itm = len(set(df['movieId']))
        model = NCF(emb_len_usr,emb_len_itm,args.embd_sz,args.dropout_p,alpha=args.alpha,with_variable_alpha=args.with_variable_alpha)
        
        device = 'cpu'
        loss_fn = nn.MSELoss()
        # Define optimizer & scheduler
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=5)
        
        
        os.makedirs(train_artifacts_loc,exist_ok=True)
        
        #train
        trainer = Trainer(model,'cpu',loss_fn,optimizer,scheduler,artifacts_loc=train_artifacts_loc,exp_tracker=exp_tracker)
        best_model, best_val_loss = trainer.train(args.num_epochs,args.patience,trn_dl,vld_dl)
        #save the model
        PATH = train_artifacts_loc/'model.pth'
        torch.save(best_model.state_dict(), PATH)
        
        test_ratings = tst_df['rating']
        #loss, np.vstack(y_trues), np.vstack(y_probs)
        tst_loss,y_trues,y_predict = trainer.eval_step(tst_dl)
        test_rslts = pd.DataFrame({'y_true':y_trues.squeeze(),'y_pred':y_predict.squeeze()},index=range(len(y_trues)))
        
        #save test predictions
        test_rslts.to_csv(str(train_artifacts_loc/('inference'+'_'+args.split_type+'_tst'+'.csv')))
        dump(args,train_artifacts_loc/'args.joblib')
        
    if args.mode == 'inference':
        losses = {}
        for split_type in args.on:
            #'trn','vld','tst'
            if split_type == 'trn':
                data = RecsysDataset(trn_df)
            elif split_type == 'vld':
                data = RecsysDataset(vld_df)
            elif split_type == 'tst':
                data = RecsysDataset(tst_df)
            data_dl = dl(data, batch_size=args.batch_size)
            #?load it from the path
            dir_loc = store_dir/(str(trail_id)+'_'+dataset+'_'+data_type)
            #model  = 
            trainer = Trainer(model,'cpu',loss_fn,optimizer,scheduler,artifacts_loc=train_artifacts_loc)
            tst_loss,y_trues,y_predict = trainer.eval_step(data_dl)
            test_rslts = pd.DataFrame({'y_true':y_trues.squeeze(),'y_pred':y_predict})
            
            losses[split_type] = tst_loss
            test_rslts.to_csv(str(dir_loc)/('inference'+'_'+split_type+'.csv'))
                
     
                
                
        
        
        
        
        
        
        



        


In [20]:
run(args)

  1%|▍                                          | 1/100 [00:00<01:32,  1.07it/s]

Epoch: 1 | train_loss: 13.95471, val_loss: 14.04778, lr: 2.00E-04, _patience: 10


 11%|████▌                                     | 11/100 [00:09<01:18,  1.13it/s]

Epoch: 11 | train_loss: 5.04894, val_loss: 4.82705, lr: 2.00E-04, _patience: 10


 21%|████████▊                                 | 21/100 [00:18<01:09,  1.13it/s]

...

 81%|██████████████████████████████████        | 81/100 [01:11<00:16,  1.12it/s]

Epoch: 81 | train_loss: 1.10881, val_loss: 1.07645, lr: 2.00E-04, _patience: 7


 91%|██████████████████████████████████████▏   | 91/100 [01:20<00:08,  1.10it/s]

Epoch: 91 | train_loss: 1.05481, val_loss: 1.02154, lr: 2.00E-04, _patience: 10


100%|█████████████████████████████████████████| 100/100 [01:29<00:00,  1.12it/s]


Above run is on a sample of the dataset(size=10000 rows). We can see that both train and validation losses is going down, thus our implementation is likely correct. We can run the same on our `full` or `partial` datasets.